In [1]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [2]:
from infrastructure.instrument_collection import instrumentCollection as ic

In [42]:
pair = "EUR_USD"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [43]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [44]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2022-05-26 21:00:00+00:00,1.07257,1.07404,1.07248,1.07401
1,2022-05-27 01:00:00+00:00,1.07399,1.07652,1.07342,1.07548
2,2022-05-27 05:00:00+00:00,1.07548,1.07571,1.07258,1.07305
3,2022-05-27 09:00:00+00:00,1.07303,1.07315,1.06968,1.07154
4,2022-05-27 13:00:00+00:00,1.07155,1.07505,1.07014,1.07102


In [45]:
for ma in MA_LIST:    
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [46]:
df_ma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3801 entries, 0 to 3800
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    3801 non-null   datetime64[ns, tzutc()]
 1   mid_o   3801 non-null   float64                
 2   mid_h   3801 non-null   float64                
 3   mid_l   3801 non-null   float64                
 4   mid_c   3801 non-null   float64                
 5   MA_10   3801 non-null   float64                
 6   MA_20   3801 non-null   float64                
 7   MA_50   3801 non-null   float64                
 8   MA_100  3801 non-null   float64                
 9   MA_200  3801 non-null   float64                
dtypes: datetime64[ns, tzutc()](1), float64(9)
memory usage: 297.1 KB


In [47]:
df_plot = df_ma.iloc[:500]

In [48]:
cp = CandlePlot(df_plot)

In [49]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [50]:
cp.show_plot(line_traces=traces)

In [51]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

# During the course you will see me use

# df.groupby(stuff....).sum()

# you should as numeric_only=True into the brackets:

# df.groupby(stuff...).sum(numeric_only=True)

In [52]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [70]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
0,2022-07-13 01:00:00+00:00,1.00304,1.00472,1.00229,1.00358,1.022835,1.049990,-0.027155,NaN,0
1,2022-07-13 05:00:00+00:00,1.00358,1.00531,1.00063,1.00366,1.021965,1.049638,-0.027673,-0.027155,0
2,2022-07-13 09:00:00+00:00,1.00364,1.01031,0.99980,1.00204,1.021098,1.049271,-0.028173,-0.027673,0
3,2022-07-13 13:00:00+00:00,1.00205,1.01224,1.00088,1.00822,1.020345,1.048947,-0.028602,-0.028173,0
4,2022-07-13 17:00:00+00:00,1.00824,1.00876,1.00540,1.00580,1.019647,1.048618,-0.028971,-0.028602,0


In [54]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [55]:
def is_trade(row):
    if row.DELTA >= 0  and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0  and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [56]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [57]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [69]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,GAIN_C
132,2022-08-12 01:00:00+00:00,1.03130,1.03218,1.03050,1.03178,1.022779,1.022581,0.000198,-0.000049,1,-0.03780,-378.0,-378.0
173,2022-08-22 21:00:00+00:00,0.99430,0.99501,0.99302,0.99398,1.016639,1.016670,-0.000031,0.000623,-1,0.01454,-145.4,-523.4
454,2022-10-26 17:00:00+00:00,1.00765,1.00888,1.00670,1.00852,0.984370,0.984238,0.000132,-0.000472,1,0.06532,653.2,129.8
918,2023-02-14 02:00:00+00:00,1.07286,1.07400,1.07255,1.07384,1.077191,1.077214,-0.000023,0.000543,-1,0.00310,-31.0,98.8
1072,2023-03-21 17:00:00+00:00,1.07680,1.07745,1.07564,1.07694,1.066944,1.066899,0.000045,-0.000433,1,0.01220,122.0,220.8


In [59]:
cp = CandlePlot(df_an.iloc[20:260])
cp.show_plot(line_traces=[MA_S, MA_L])

In [60]:
ic.LoadInstruments("../data")

In [61]:
ic.instruments_dict[pair]

{'name': 'EUR_USD', 'ins_type': 'CURRENCY', 'displayName': 'EUR/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.02, 'displayPrecision': 5}

In [62]:
ins_data = ic.instruments_dict[pair]

In [63]:
df_trades.shape

(16, 10)

In [64]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [65]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [66]:
df_trades.GAIN.sum()

np.float64(752.1000000000022)

In [67]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [68]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])